# Data Wrapper

In [1]:
# Imports
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.linear_model import Lasso, Ridge, LassoCV, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.decomposition import PCA

# EDA
import missingno

from collections import defaultdict
import re
import math

import matplotlib.pyplot as plt
import matplotlib as matplot
from matplotlib import cm
import seaborn as sns
import itertools

%matplotlib inline
# Matplotlib params
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('ggplot')

# Pandas display options
pd.set_option('display.max_columns',6000)
pd.set_option('display.max_rows',6000)

import numpy as np
import tqdm

from collections import defaultdict

# show all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import

In [2]:
df = pd.read_csv('Files/Activities/activities.csv',sep = ';')

In [5]:
df.sample(3).drop(['representative','organization_name','description','name','auth_token','edit_url','email'],axis=1)

,id,activity_category_id,activity_category,meetup_point,meetup_date,street,street_number,city,postal_code,longitude,latitude,approved,created_at,photo_ids
1332,4544,1,Δράση Κυριακής,ΣΤΙΡΦΑΚΑ,2019-04-06 10:00:00 +0300,NaN,NaN,NaN,NaN,22.306856,38.954502,True,2019-04-05 14:14:02 +0300,[]
614,3808,1,Δράση Κυριακής,ΠΑΙΔΙΚΗ ΧΑΡΑ ΣΤΟ ΠΥΡΟΒΟΛΙΚΟ,2019-04-07 11:00:00 +0300,ΠΑΙΔΙΚΗ ΧΑΡΑ ΣΤΟ ΠΥΡΟΒΟΛΙΚΟ,ΔΗΜΗΤΡΙΟΥ ΚΡΟΚΟΥ,ΠΡΕΒΕΖΑ,48100,20.747378,38.959562,True,2019-03-27 08:13:54 +0200,[]
1155,4361,2,Δράση Εβδομάδας Εθελοντισμού στα Σχολεία,Δημοτικό Σχολείο Σίφνου,2019-04-04 08:00:00 +0300,NaN,NaN,NaN,NaN,24.714111,36.976000,True,2019-04-03 16:16:51 +0300,[]


## Convert to GeoDataFrame

In [7]:
import geopandas as gpd
import shapefile
from shapely.geometry import Point
from shapely.geometry import shape
from pyproj import Proj, transform

In [8]:
from geopy.distance import geodesic

In [9]:
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.longitude, df.latitude),crs ="+init=EPSG:4326" )

## Load shapefiles

In [10]:
# Import shape
shp_m = shapefile.Reader('Files/Shapefiles/Municipalities/oria_dhmwn_kallikraths.dbf')
shp_r = shapefile.Reader('Files/Shapefiles/Regions/periphereies.dbf')
shp_d = shapefile.Reader('Files/Shapefiles/Region_districts/ΠΕΡΙΦΕΡΕΙΑΚΕΣ_ΕΝΟΤΗΤΕΣ.dbf')

### Create gdf from shapefiles

In [11]:
reg = gpd.GeoDataFrame(pd.DataFrame(shp_r.records(),columns = ['Name']), geometry = shp_r.shapes(),crs = "+init=EPSG:2100")
mun = gpd.GeoDataFrame(pd.DataFrame(shp_m.records(),columns = ['Name','Code']), geometry = shp_m.shapes(), crs = "+init=EPSG:2100")
rdi = gpd.GeoDataFrame(pd.DataFrame(pd.DataFrame(shp_d.records())[2].values,columns = ['Name']), geometry = shp_d.shapes(), crs = "+init=EPSG:2100")

In [12]:
reg.crs = {'init' :'epsg:2100'}

### Transform shapefiles to global coordinate system

In [13]:
reg = reg.to_crs(epsg=4326)
mun = mun.to_crs(gdf.crs)
rdi = rdi.to_crs(epsg=4326)

# Get municipality / region name for each event

In [14]:
for i in tqdm.tqdm(reg.index):
    poly = reg.loc[i,'geometry']
    name =  reg.loc[i,'Name']
    gdf.loc[gdf.within(poly),'Region'] = name

100%|██████████| 13/13 [00:33<00:00,  2.58s/it]


In [15]:
for i in tqdm.tqdm(mun.index):
    poly = mun.loc[i,'geometry']
    name =  mun.loc[i,'Name']
    gdf.loc[gdf.within(poly),'Municipality'] = name

100%|██████████| 326/326 [00:14<00:00, 22.01it/s]


In [16]:
for i in tqdm.tqdm(rdi.index):
    poly = rdi.loc[i,'geometry']
    name =  rdi.loc[i,'Name']
    gdf.loc[gdf.within(poly),'Subregion'] = name

100%|██████████| 75/75 [00:14<00:00,  5.10it/s]


In [17]:
gdf.loc[gdf['Region'].isna(),:].shape
gdf.loc[gdf['Subregion'].isna(),:].shape

(76, 25)

(83, 25)

In [18]:
pnt = gdf.loc[gdf['Region'].isna(),:].head(1).loc[3,'geometry']

In [19]:
from shapely.ops import nearest_points
from shapely.geometry import MultiPoint

In [20]:
def get_min_distance(point,regions):
    """Get minimum distance and reference region.
    
    point: point to get distance from (shapely.Point)
    region: Gdf to get distances from. Should have columns [geometry,Name]"""
    
    reg = regions.copy()
    
    reg['Nearest Point'] = reg['geometry'].apply(lambda x: nearest_points(point,x)[-1])
    reg['Distance'] = reg['Nearest Point'].apply(lambda X: geodesic(tuple((point.y,point.x)),tuple((X.y,X.x))))
  
    return reg

In [21]:
for ind in tqdm.tqdm(gdf.loc[gdf['Region'].isna(),:].index):
    pnt = gdf.loc[ind,'geometry']
    
    # Get Region
    reg_dist = get_min_distance(pnt,reg)
    gdf.loc[ind,'Region'] = reg_dist.sort_values(by = 'Distance',ascending = True)['Name'].iloc[0]
    
for ind in tqdm.tqdm(gdf.loc[gdf['Municipality'].isna(),:].index):
    # Get Municipality
    mun_dist = get_min_distance(pnt,mun)
    gdf.loc[ind,'Municipality'] = mun_dist.sort_values(by = 'Distance',ascending = True)['Name'].iloc[0]
    
for ind in tqdm.tqdm(gdf.loc[gdf['Subregion'].isna(),:].index):
    # Get Municipality
    rdi_dist = get_min_distance(pnt,rdi)
    gdf.loc[ind,'Subregion'] = rdi_dist.sort_values(by = 'Distance',ascending = True)['Name'].iloc[0]

100%|██████████| 83/83 [00:03<00:00, 27.02it/s]


# Convert regions - municipalities

In [22]:
from shapely.geometry import Polygon

def multipolygon_to_polygons(multi):
    """get polygons out of a multipolygon.
    returns a list of polygons"""    
    poly = [Polygon(line) for line in multi.boundary]
    
    return poly

In [23]:
def series_multipolygons_to_gdf(gs,crs):
    """transforms a geoseries witha multipolygon to a geodataframe of single polygons.
    All other columns of the geoseries remain the same.
    
    gs: GeoSeries
    crs: reference crs""" 
    
    # Get list of polygons
    poly = multipolygon_to_polygons(gs['geometry'])

    # Create gdf with the same crs
    g = gpd.GeoDataFrame({'geometry':poly},crs =crs)
    
    # Get all other columns
    for col in gs.drop('geometry').index:
        g[col] = gs[col]
        
    return g

## Regions

In [24]:
reg['geometry'] = reg['geometry'].simplify(0.005, preserve_topology=False)

In [25]:
reg_poly = gpd.GeoDataFrame(crs = reg.crs)

for i in tqdm.tqdm(reg.index):
    if reg.loc[i,'geometry'].type == 'MultiPolygon':
        temp = series_multipolygons_to_gdf(reg.loc[i],reg_poly.crs)
    else:
        temp = reg.loc[[i]]
    
    reg_poly = pd.concat([reg_poly,temp],axis=0,sort=False)

100%|██████████| 13/13 [00:00<00:00, 290.27it/s]


### Check minimum areas

In [26]:
reg['geometry'].apply(lambda x: x.area).describe()
mun['geometry'].apply(lambda x: x.area).describe()

count    13.000000
mean      1.058029
std       0.566296
min       0.238846
25%       0.534040
50%       1.002254
75%       1.523963
max       2.047869
Name: geometry, dtype: float64

count    326.000000
mean       0.042161
std        0.037853
min        0.000215
25%        0.007782
50%        0.037395
75%        0.062692
max        0.196472
Name: geometry, dtype: float64

The smallest municipality (probably Gavdos) has an area of 0.000215.

In [27]:
# Reset index
reg_poly.reset_index(inplace=True,drop=True)
# Get area
reg_poly['area'] = reg_poly['geometry'].apply(lambda x: x.area)

# Set benchmark: mean - 1.5std
benchmark = mun['geometry'].apply(lambda x: x.area).min() # area of the smallest municipality
reg_poly_small = reg_poly.loc[reg_poly['area'] > benchmark]

In [28]:
reg_poly.shape
reg_poly_small.shape

(252, 3)

(146, 3)

We reduced the polygons from 2k to 153.

## Municipalities

In [29]:
mun['geometry'] = mun['geometry'].simplify(0.005, preserve_topology=False)

In [30]:
mun_poly = gpd.GeoDataFrame(crs = mun.crs)

for i in tqdm.tqdm(mun.index):
    if mun.loc[i,'geometry'].type == 'MultiPolygon':
        temp = series_multipolygons_to_gdf(mun.loc[i],mun_poly.crs)
    else:
        temp = mun.loc[[i]]
    
    mun_poly = pd.concat([mun_poly,temp],axis=0,sort=False)

100%|██████████| 326/326 [00:00<00:00, 552.68it/s]


In [31]:
# Reset index
mun_poly.reset_index(inplace=True,drop=True)

# Get area
mun_poly['area'] = mun_poly['geometry'].apply(lambda x: x.area)

# Set benchmark: mean - 1.5std
benchmark = mun['geometry'].apply(lambda x: x.area).min() # area of the smallest municipality
mun_poly = mun_poly.loc[mun_poly['area'] > benchmark]

## Subregions

In [32]:
rdi['geometry'] = rdi['geometry'].simplify(0.005, preserve_topology=False)

In [33]:
rdi_poly = gpd.GeoDataFrame(crs = rdi.crs)

for i in tqdm.tqdm(rdi.index):
    if rdi.loc[i,'geometry'].type == 'MultiPolygon':
        temp = series_multipolygons_to_gdf(rdi.loc[i],rdi_poly.crs)
    else:
        temp = rdi.loc[[i]]
    
    rdi_poly = pd.concat([rdi_poly,temp],axis=0,sort=False)

100%|██████████| 75/75 [00:00<00:00, 449.20it/s]


In [34]:
# Reset index
rdi_poly.reset_index(inplace=True,drop=True)

# Get area
rdi_poly['area'] = rdi_poly['geometry'].apply(lambda x: x.area)

# Set benchmark: mean - 1.5std
benchmark = mun['geometry'].apply(lambda x: x.area).min() # area of the smallest municipality
rdi_poly = rdi_poly.loc[rdi_poly['area'] > benchmark]

# Get final df

In [35]:
df = pd.DataFrame(gdf.drop('geometry',axis=1),index = gdf.index)

# Delete those that are on the default spot

In [36]:
filter_default = (df['longitude'].round(4) == 24.5394 ) & (df['latitude'].round(4) == 38.0995)

In [37]:
df.loc[filter_default,['Region','Municipality','Subregion']] = np.nan

# Export

In [38]:
df.to_pickle('Files/Output/activites_for_map_data.pkl')
gdf[['id','geometry']].to_file('Files/Output/activities_for_map_geometry.pkl')

In [39]:
reg_poly_small.to_file('Files/Output/regions_polygons.pkl')
mun_poly.to_file('Files/Output/municipalities_polygons.pkl')
rdi_poly.to_file('Files/Output/subregions_polygons.pkl')